In [1]:
!pip install transformers==4.46.2 datasets==3.1.0 torch==2.5.1+cu121 sklearn-pandas==2.2.0 numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
import datasets
import torch
import os

The WANDB_API_KEY is nessesary to use the Trainer class from HuggingFace and is what's used in the example notebook. However, I think we may all have to put in our own api key secret into google colab for it to work.

In [3]:
# from google.colab import userdata
# os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

In [4]:
# langs = ['java', 'python', 'pharo']

langs = ['java'] # Using Java as the only language for testing purposes.
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = datasets.load_dataset('NLBSE/nlbse25-code-comment-classification')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

java_train-00000-of-00001.parquet:   0%|          | 0.00/680k [00:00<?, ?B/s]

java_test-00000-of-00001.parquet:   0%|          | 0.00/174k [00:00<?, ?B/s]

python_train-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

...

Generating java_test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Generating python_train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

Generating python_test split:   0%|          | 0/406 [00:00<?, ? examples/s]

Generating pharo_train split:   0%|          | 0/1298 [00:00<?, ? examples/s]

Generating pharo_test split:   0%|          | 0/289 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [6]:
ds['java_test'][0]

{'index': 5,
 'class': 'AbstractContractGetFileStatusTest.java',
 'comment_sentence': 'accept everything.',
 'partition': 1,
 'combo': 'accept everything. | AbstractContractGetFileStatusTest.java',
 'labels': [0, 0, 1, 0, 0, 0, 0]}

In [7]:
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

The next few functions are to preprocess the trainng and validation sets, and to let the Trainer class to evaluate how well the trainng is going after each epoch.

In [8]:
# To tokenize the text in the 'combo' column of the training dataset of each language.

def tokenize_dataset(examples):
    return tokenizer(examples['combo'], truncation=True, padding="max_length", max_length=128)

In [9]:
# Tokenizes the text 'combo' column of the dataset, changes the values of the
# labels column to float instead of int, and sets to format of the lables column
# to torch tesnsors, which is required by the Trainer class.

def preprocess_dataset(input_dataset):
  processed_dataset = input_dataset.map(tokenize_dataset, batched=True, load_from_cache_file=False)
  processed_dataset = processed_dataset.cast_column("labels", datasets.features.Sequence(datasets.features.Value("float32")))
  processed_dataset.set_format('pt')
  return processed_dataset

In [10]:
# To compute eveluation metrics during the training process.

def compute_metrics(pred):
    logits, labels = pred
    preds = torch.sigmoid(torch.tensor(logits)) > 0.5
    labels = torch.tensor(labels)

    precision = precision_score(labels, preds, average='micro')
    recall = recall_score(labels, preds, average='micro')
    f1 = f1_score(labels, preds, average='micro')

    return {"precision": precision, "recall": recall, "f1": f1}

In [11]:
# Training the models for each language and saving the the models and thier
# tokenizers after training.

for lang in langs:
  num_labels = len(labels[lang])
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, problem_type="multi_label_classification")

  dataset = preprocess_dataset(ds[f'{lang}_train'])
  train_valalidation_split = dataset.train_test_split(test_size=0.2, seed=27)

  train_dataset =  train_valalidation_split['train']
  validation_dataset = train_valalidation_split['test']

  training_args = TrainingArguments(
    output_dir=f'./results_{lang}',
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=f'./logs_{lang}',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=5e-5,
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
  )

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset= validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  )

  trainer.train()
  metrics = trainer.evaluate()

  print("Evaluation Metrics:", metrics)

  trainer.model.save_pretrained(f'./models/{lang}')
  tokenizer.save_pretrained(f'./tokenizers/{lang}')

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7614 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7614 [00:00<?, ? examples/s]

<ipython-input-11-061f5d1d54dd>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.073400,0.089892,0.927736,0.860792,0.893011
2,0.101500,0.081014,0.927815,0.894636,0.910923
3,0.053300,0.079725,0.930492,0.906130,0.918149


Evaluation Metrics: {'eval_loss': 0.0797247514128685, 'eval_precision': 0.9304918032786885, 'eval_recall': 0.9061302681992337, 'eval_f1': 0.9181494661921709, 'eval_runtime': 9.9537, 'eval_samples_per_second': 153.009, 'eval_steps_per_second': 19.189, 'epoch': 3.0}


Evaluating the predictions.

In [21]:
import pandas as pd
import time

In [22]:
# Reloading the saved model for java, and it's tokenizer

model = AutoModelForSequenceClassification.from_pretrained('./models/java')
tokenizer = AutoTokenizer.from_pretrained('./tokenizers/java')

In [23]:
# Getting sample text from the test dataset for Java

sample_text = ds['java_test']['combo'][0]

In [24]:
sample_text

'accept everything. | AbstractContractGetFileStatusTest.java'

In [25]:
# As far as I understand, CodeBERT can only make predictions on tokenized
# text that also has an attention mask. To get that for the input text, we have to
# encode it using the tokenizer and convert it to a torch tesnor.

encoded_test = tokenizer(sample_text, return_tensors="pt")

In [26]:
# The encoded sample text with it's attention mask.

encoded_test

{'input_ids': tensor([[    0, 35468,   960,     4,  1721, 43649, 43218, 14181,  9966, 47731,
         34603,     4, 43830,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [27]:
results = model(**encoded_test)

In [28]:
# The logits tensor shows what the predicted results are, with the more positive
# numbers indecating the lables the model has predicted with the highest probablility.

results

SequenceClassifierOutput(loss=None, logits=tensor([[-4.3638, -5.7541, -0.2655, -3.3848, -0.9917, -3.4193, -1.4983]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [29]:
# These are the actual results for that set in the lables column.

ds['java_test']['labels'][0]

[0, 0, 1, 0, 0, 0, 0]

Below is non-functional code I still haven't finnished. We need to implement a way for the logits returned from the model to be converted into binary (0 - 1) predictions for the labels, and for those labels to be compaired to the ground truth labels of the testing datasets for each language. The competition also wants us to complete this step using profiling that they have set up, and which is partially shown below. The link to the reference notebook for the competition is [here](https://colab.research.google.com/drive/1RULzasjO_nrqiXLrGze-PznFlHKtGQ4s?usp=sharing). Also, another notebook that is fairly close to what I believe we are trying to do can be found [here](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb).

In [ ]:
# total_flops = 0
# total_time = 0
# scores = []
# for lan in langs:
#     # to load trained models:
#     model = AutoModelForSequenceClassification.from_pretrained(f'./models/{lan}')
#     # to load pretrained models from Hub:
#     #model = SetFitModel.from_pretrained(f"NLBSE/nlbse25_{lan}")
#     with torch.profiler.profile(with_flops=True) as p:
#         begin = time.time()
#         for i in range(10):
#           y_pred = model(ds[f'{lan}_test']['combo']).numpy().T
#         total = time.time() - begin
#         total_time = total_time + total
#     total_flops = total_flops + (sum(k.flops for k in p.key_averages()) / 1e9)
#     y_true = np.array(ds[f'{lan}_test']['labels']).T
#     for i in range(len(y_pred)):
#         assert(len(y_pred[i]) == len(y_true[i]))
#         tp = sum([true == pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
#         tn = sum([true == pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
#         fp = sum([true == 0 and pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
#         fn = sum([true == 1 and pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
#         precision = tp / (tp + fp)
#         recall = tp / (tp + fn)
#         f1 = (2*tp) / (2*tp + fp + fn)
#         scores.append({'lan': lan, 'cat': labels[lan][i],'precision': precision,'recall': recall,'f1': f1})
# print("Compute in GFLOPs:", total_flops/10)
# print("Avg runtime in seconds:", total_time/10)
# scores = pd.DataFrame(scores)

In [ ]:
# scores

In [ ]:
# max_avg_runtime = 5
# max_avg_flops = 5000

# # s𝑢𝑏𝑚𝑖𝑠𝑠𝑖𝑜𝑛_𝑠𝑐𝑜𝑟𝑒(𝑚𝑜𝑑𝑒𝑙)=(𝑎𝑣𝑔. 𝐹1)×0.60+((𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)/𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)×0.2+((𝑚𝑎𝑥_GFLOPs−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_GFLOPs)/𝑚𝑎𝑥_GFLOPs)×0.2
# def score(avg_f1, avg_runtime, avg_flops):
#     return (0.6 * avg_f1 +
#       0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
#       0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

# avg_f1 = scores.f1.mean()
# avg_runtime = total_time/10
# avg_flops = total_flops/10

# round(score(avg_f1, avg_runtime, avg_flops), 2)